## 1. Importing data for confirmed cases and deaths

In [1]:
import pandas as pd

df_confirmed = pd.read_csv("raw_data/us_confirmed.csv")

df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538065 entries, 0 to 538064
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   UID             538065 non-null  int64  
 1   iso2            538065 non-null  object 
 2   iso3            538065 non-null  object 
 3   code3           538065 non-null  int64  
 4   FIPS            536415 non-null  float64
 5   Admin2          536910 non-null  object 
 6   Lat             538065 non-null  float64
 7   Combined_Key    538065 non-null  object 
 8   Date            538065 non-null  object 
 9   Case            538065 non-null  int64  
 10  Long            538065 non-null  float64
 11  Country/Region  538065 non-null  object 
 12  Province/State  538065 non-null  object 
dtypes: float64(3), int64(3), object(7)
memory usage: 53.4+ MB


In [2]:
df_deaths = pd.read_csv("raw_data/us_deaths.csv")

df_deaths.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Lat,Combined_Key,Population,Date,Case,Long,Country/Region,Province/State
0,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-22,0,-170.132,US,American Samoa
1,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-23,0,-170.132,US,American Samoa
2,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-24,0,-170.132,US,American Samoa
3,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-25,0,-170.132,US,American Samoa
4,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-26,0,-170.132,US,American Samoa


## 2. Subset data to first 100 cases

In [3]:
df_first_hundred = df_confirmed.loc[df_confirmed["Case"] >= 100].drop_duplicates(subset=["Province/State"], keep="first")
df_first_hundred = df_first_hundred[["Date","Case", "Province/State"]].reset_index(drop=True)
df_first_hundred.columns = ["Date","Case", "State"]

df_first_hundred

,Date,Case,State
0,2020-04-05,112,Guam
1,2020-03-28,100,Puerto Rico
2,2020-07-04,111,Virgin Islands
3,2020-05-14,104,Alabama
4,2020-04-08,103,Alaska
5,2020-04-15,110,Arizona
6,2020-04-29,101,Arkansas
7,2020-03-23,118,California
8,2020-03-29,110,Colorado
9,2020-03-19,102,Connecticut


## 3. Importing policy data

- subset dataframe for "PublicMask" and "StayAtHome" policies
- find dates for when these policies were enacted


In [14]:
df_policies = pd.read_csv("raw_data/USstatesCov19distancingpolicy.csv", delimiter=";")

In [5]:
# Subsetting for PublicMask and StayAtHome

df_policies = df_policies[df_policies["StatePolicy"].isin(["PublicMask", "StayAtHome"])]


In [16]:
# Separating df into PublicMask and StayAtHome

df_public_mask = df_policies.loc[df_policies["StatePolicy"] == "PublicMask"].reset_index(drop=True)

df_public_mask

,location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,DateIssued,DateEnacted,DateExpiry,DateEased,DateEnded,DateReexpanded1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes
0,523,1,AL,Alabama,PublicMask,0,1,20200428,20200430,20200731.0,NaN,NaN,NaN,Recommendation from the Gov in the Safer At Ho...,http://www.alabamapublichealth.gov/legal/asset...,20200701,Updated DateExpiry to 20200731
1,524,2,AK,Alaska,PublicMask,0,1,20200403,20200403,NaN,NaN,NaN,NaN,Highly recommended for all Alaskans: wear a cl...,http://dhss.alaska.gov/dph/Epi/id/SiteAssets/P...,20200426,NaN
2,526,5,AR,Arkansas,PublicMask,0,1,20200423,20200423,NaN,NaN,NaN,NaN,"Everyone should wear a face covering, or mask,...",https://www.healthy.arkansas.gov/images/upload...,20200623,Updated PolicyNotes re extension
3,527,6,CA,California,PublicMask,0,1,20200401,20200401,NaN,NaN,NaN,NaN,Provides guidance around using face coverings ...,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...,20200525,NaN
4,527,6,CA,California,PublicMask,1,1,20200618,20200618,NaN,NaN,NaN,NaN,(quote) People in California must wear face co...,https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH...,20200621,NaN
5,528,8,CO,Colorado,PublicMask,0,1,20200426,20200427,20200720.0,NaN,NaN,NaN,(quote) Individuals are urged to wear non-medi...,https://drive.google.com/file/d/1l13eSver8iS1f...,20200623,UpdatedDate Expiry to 20200720
6,529,9,CT,Connecticut,PublicMask,1,1,20200417,20200420,NaN,NaN,NaN,NaN,Masks or face coverings required of all people...,https://portal.ct.gov/-/media/Office-of-the-Go...,20200422,NaN
7,530,10,DE,Delaware,PublicMask,1,1,20200425,20200428,NaN,NaN,NaN,NaN,All individuals over two who are medically abl...,https://governor.delaware.gov/health-soe/thirt...,20200623,Noted extension
8,531,11,DC,District of Columbia,PublicMask,1,1,20200415,20200417,20200725.0,20200622.0,NaN,NaN,(quote) DOH guidance relating to masks must be...,https://coronavirus.dc.gov/sites/default/files...,20200629,Updated DateExpiry to 20200725
9,531,11,DC,District of Columbia,PublicMask,0,1,20200619,20200622,NaN,NaN,NaN,NaN,Eased to recommendation under Phase 2 reopenin...,https://coronavirus.dc.gov/phasetwo,20200624,NaN


In [19]:
df_stay_home = df_policies.loc[df_policies["StatePolicy"] == "StayAtHome"]
df_stay_home["StateName"].value_counts()

Pennsylvania            8
Idaho                   3
Tennessee               3
Louisiana               3
Mississippi             3
Kansas                  2
Oklahoma                2
Minnesota               2
Vermont                 2
Colorado                2
Ohio                    2
Texas                   2
New Hampshire           2
Alaska                  2
Utah                    2
Georgia                 2
Alabama                 2
North Carolina          2
Indiana                 2
Florida                 2
South Carolina          2
West Virginia           2
Delaware                2
Illinois                2
Missouri                1
New Jersey              1
Kentucky                1
Connecticut             1
New York                1
Massachusetts           1
South Dakota            1
Arizona                 1
Oregon                  1
Maine                   1
Hawaii                  1
Wisconsin               1
New Mexico              1
Washington              1
Montana     

In [20]:
# subset for dates when policies were enacted

df_public_mask = df_public_mask[["StateName","Mandate", "StateWide", "DateEnacted"]]
df_stay_home = df_stay_home[["StateName","Mandate", "StateWide", "DateEnacted"]]


In [21]:
# Changing "DateEnacted" to "PublicMask_date" and converting to datetime

df_public_mask.columns = ["State","Mandate_mask", "StateWide_mask", "PublicMask_date"]

df_public_mask["PublicMask_date"] = pd.to_datetime(df_public_mask["PublicMask_date"], format="%Y%m%d")

df_public_mask = df_public_mask.drop_duplicates(subset=["State"], keep="first")

df_public_mask

<ipython-input-21-c2ad08b45c76>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_public_mask["PublicMask_date"] = pd.to_datetime(df_public_mask["PublicMask_date"], format="%Y%m%d")


,State,Mandate_mask,StateWide_mask,PublicMask_date
0,Alabama,0,1,2020-04-30
1,Alaska,0,1,2020-04-03
2,Arkansas,0,1,2020-04-23
3,California,0,1,2020-04-01
5,Colorado,0,1,2020-04-27
6,Connecticut,1,1,2020-04-20
7,Delaware,1,1,2020-04-28
8,District of Columbia,1,1,2020-04-17
10,Florida,0,1,2020-06-20
11,Georgia,0,1,2020-05-01


In [22]:
# Changing "DateEnacted" to "StayAtHome_date" and converting to datetime

df_stay_home.columns = ["State","Mandate_home", "StateWide_home", "StayAtHome_date"]
df_stay_home["StayAtHome_date"] = pd.to_datetime(df_stay_home["StayAtHome_date"], format="%Y%m%d")

df_stay_home = df_stay_home.drop_duplicates(subset=["State"], keep="first").reset_index(drop=True)

df_stay_home

,State,Mandate_home,StateWide_home,StayAtHome_date
0,Alabama,1,1,2020-04-04
1,Alaska,1,0,2020-03-22
2,Arizona,1,1,2020-03-31
3,California,1,1,2020-03-19
4,Colorado,1,1,2020-03-26
5,Connecticut,0,1,2020-03-23
6,Delaware,1,1,2020-03-24
7,District of Columbia,1,1,2020-04-01
8,Florida,1,1,2020-04-03
9,Georgia,1,0,2020-03-24


## 4. Merge policy date dataframes

- merge dataframes
- subtract policy dates from dates of first 100 cases per state

In [32]:
df_mask_and_stay_home = df_public_mask.merge(df_stay_home, how="outer")
df_mask_and_stay_home

,State,Mandate_mask,StateWide_mask,PublicMask_date,Mandate_home,StateWide_home,StayAtHome_date
0,Alabama,0.0,1.0,2020-04-30,1.0,1.0,2020-04-04
1,Alaska,0.0,1.0,2020-04-03,1.0,0.0,2020-03-22
2,Arkansas,0.0,1.0,2020-04-23,NaN,NaN,NaT
3,California,0.0,1.0,2020-04-01,1.0,1.0,2020-03-19
4,Colorado,0.0,1.0,2020-04-27,1.0,1.0,2020-03-26
5,Connecticut,1.0,1.0,2020-04-20,0.0,1.0,2020-03-23
6,Delaware,1.0,1.0,2020-04-28,1.0,1.0,2020-03-24
7,District of Columbia,1.0,1.0,2020-04-17,1.0,1.0,2020-04-01
8,Florida,0.0,1.0,2020-06-20,1.0,1.0,2020-04-03
9,Georgia,0.0,1.0,2020-05-01,1.0,0.0,2020-03-24


In [37]:
df_final = (df_first_hundred.merge(df_mask_and_stay_home, on="State", how="left")
            .rename({"Date":"Date first 100 cases", "Case":"Number of cases => 100"}, axis="columns"))

df_final

,Date first 100 cases,Number of cases => 100,State,Mandate_mask,StateWide_mask,PublicMask_date,Mandate_home,StateWide_home,StayAtHome_date
0,2020-04-05,112,Guam,NaN,NaN,NaT,NaN,NaN,NaT
1,2020-03-28,100,Puerto Rico,NaN,NaN,NaT,NaN,NaN,NaT
2,2020-07-04,111,Virgin Islands,NaN,NaN,NaT,NaN,NaN,NaT
3,2020-05-14,104,Alabama,0.0,1.0,2020-04-30,1.0,1.0,2020-04-04
4,2020-04-08,103,Alaska,0.0,1.0,2020-04-03,1.0,0.0,2020-03-22
5,2020-04-15,110,Arizona,NaN,NaN,NaT,1.0,1.0,2020-03-31
6,2020-04-29,101,Arkansas,0.0,1.0,2020-04-23,NaN,NaN,NaT
7,2020-03-23,118,California,0.0,1.0,2020-04-01,1.0,1.0,2020-03-19
8,2020-03-29,110,Colorado,0.0,1.0,2020-04-27,1.0,1.0,2020-03-26
9,2020-03-19,102,Connecticut,1.0,1.0,2020-04-20,0.0,1.0,2020-03-23


In [40]:
df_final["Date first 100 cases"] = pd.to_datetime(df_final["Date first 100 cases"])

In [51]:
df_final = df_final.assign(StayAtHome_Delta = pd.to_timedelta(np.where(df_final["StayAtHome_date"].notna(), 
                                                        df_final["StayAtHome_date"] - df_final["Date first 100 cases"], 
                                                        pd.NA)))

In [52]:
df_final = df_final.assign(PublicMask_Delta = pd.to_timedelta(np.where(df_final["StayAtHome_date"].notna(), 
                         df_final["PublicMask_date"] - df_final["Date first 100 cases"], 
                         pd.NA)))

In [54]:
df_final.head()

,Date first 100 cases,Number of cases => 100,State,Mandate_mask,StateWide_mask,PublicMask_date,Mandate_home,StateWide_home,StayAtHome_date,StayAtHome_Delta,PublicMask_Delta
0,2020-04-05,112,Guam,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaT
1,2020-03-28,100,Puerto Rico,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaT
2,2020-07-04,111,Virgin Islands,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaT
3,2020-05-14,104,Alabama,0.0,1.0,2020-04-30,1.0,1.0,2020-04-04,-40 days,-14 days
4,2020-04-08,103,Alaska,0.0,1.0,2020-04-03,1.0,0.0,2020-03-22,-17 days,-5 days


In [55]:
df_final.to_csv("data/mask_and_lockdown_policy_data.csv")